In [1]:
import os
import json
from typing import Dict, List, Optional, Union, cast
from requests import get
from bs4 import BeautifulSoup
import time
import acquire
import acquire_df
from acquire import scrape_github_data
import pandas as pd
import unicodedata
import re
import json
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
import pandas as pd
import acquire
from env import github_token, github_username

In [2]:
df = acquire_df.get_NLP_df()
df

repo      language  \
0            dickreuter/Poker        Python   
1                pret/pokered      Assembly   
2      ishikota/PyPokerEngine        Python   
3      andrewprock/pokerstove           C++   
4           pokerregion/poker        Python   
..                        ...           ...   
495  IMGameDesigner/PokerGame    JavaScript   
496           houen/PokerFace  ActionScript   
497        ngocbd/pokerserver          Java   
498  mikaelbr/AI-Poker-Player        Python   
499    ProPanek/PlanningPoker    JavaScript   

                                       readme_contents  
0    Deep mind pokerbot for pokerstars, partypoker ...  
1    # Pokémon Red and Blue [![Build Status][ci-bad...  
2    # PyPokerEngine\n\n[![Build Status](https://tr...  
3    PokerStove\n==========\n\n[![OpenHUB Metrics](...  
4    Poker framework for Python\n==================...  
..                                                 ...  
495  ### 满意请给个star\n### [1.0原版js](https://github.co...  
496  This [Actionscript Flash poker (5-card & Texas...  
497  # pokerserver\n[![Codacy Badge](https://api.co...  
498  # AI Poker Player\n===========================...  
499  # Planning Poker\n[![Codacy Badge](https://api...  

[500 rows x 3 columns]

In [3]:
df = df.drop_duplicates(subset='readme_contents', keep='first', inplace=False, ignore_index=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 456 entries, 0 to 499
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   repo             456 non-null    object
 1   language         440 non-null    object
 2   readme_contents  456 non-null    object
dtypes: object(3)
memory usage: 14.2+ KB


In [4]:
# Now that I have a dataframe I need to normalize the text using common tools from previous functions to prep the text for parsing

cleaned_df = df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

cleaned_df.info()

# DataFrame.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 440 entries, 0 to 499
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   repo             440 non-null    object
 1   language         440 non-null    object
 2   readme_contents  440 non-null    object
dtypes: object(3)
memory usage: 13.8+ KB


In [5]:
cleaned_df.isna().value_counts()

repo   language  readme_contents
False  False     False              440
dtype: int64

In [6]:
cleaned_df.dtypes

repo               object
language           object
readme_contents    object
dtype: object

In [7]:
cleaned_df.readme_contents.astype("str")

0      Deep mind pokerbot for pokerstars, partypoker ...
1      # Pokémon Red and Blue [![Build Status][ci-bad...
2      # PyPokerEngine\n\n[![Build Status](https://tr...
3      PokerStove\n==========\n\n[![OpenHUB Metrics](...
4      Poker framework for Python\n==================...
                             ...                        
495    ### 满意请给个star\n### [1.0原版js](https://github.co...
496    This [Actionscript Flash poker (5-card & Texas...
497    # pokerserver\n[![Codacy Badge](https://api.co...
498    # AI Poker Player\n===========================...
499    # Planning Poker\n[![Codacy Badge](https://api...
Name: readme_contents, Length: 440, dtype: object

In [8]:
cleaned_df = cleaned_df.reset_index()
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440 entries, 0 to 439
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   index            440 non-null    int64 
 1   repo             440 non-null    object
 2   language         440 non-null    object
 3   readme_contents  440 non-null    object
dtypes: int64(1), object(3)
memory usage: 13.9+ KB


In [9]:
prepared_df = acquire_df.prepare_poker(cleaned_df.readme_contents)
prepared_df.head()

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/frederik.lindsey/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/frederik.lindsey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/frederik.lindsey/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/frederik.lindsey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/frederik.lindsey/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/frederik.lindsey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/frederik.lindsey/nltk_data..

content  \
0  Deep mind pokerbot for pokerstars, partypoker ...   
1  # Pokémon Red and Blue [![Build Status][ci-bad...   
2  # PyPokerEngine\n\n[![Build Status](https://tr...   
3  PokerStove\n==========\n\n[![OpenHUB Metrics](...   
4  Poker framework for Python\n==================...   

                                     stemmed_content  \
0  deep mind pokerbot pokerstar partypok ggpoker ...   
1  pokemon red blue build statuscibadgeci disasse...   
2  pypokerengin build statushttpstravisciorgishik...   
3  pokerstov openhub metricshttpswwwopenhubnetppo...   
4  poker framework python travi coveral python re...   

                                      lemmed_content  
0  deep mind pokerbot pokerstars partypoker ggpok...  
1  pokemon red blue build statuscibadgeci disasse...  
2  pypokerengine build statushttpstravisciorgishi...  
3  pokerstove openhub metricshttpswwwopenhubnetpp...  
4  poker framework python travis coverall python ...

In [10]:
# now merge the cleaned DF and the original DF, so that we have access to the repo titles and programming language from the original DF 
# as well as the cleaned outputs from the second DF

labeled_df = cleaned_df.merge(prepared_df, how='left', left_on='readme_contents', right_on='content')
#labeled_df = labeled_df.drop(columns='readme_contents')
labeled_df


#DataFrame.merge(right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False, validate=None)

index                      repo      language  \
0        0          dickreuter/Poker        Python   
1        1              pret/pokered      Assembly   
2        2    ishikota/PyPokerEngine        Python   
3        3    andrewprock/pokerstove           C++   
4        4         pokerregion/poker        Python   
..     ...                       ...           ...   
435    495  IMGameDesigner/PokerGame    JavaScript   
436    496           houen/PokerFace  ActionScript   
437    497        ngocbd/pokerserver          Java   
438    498  mikaelbr/AI-Poker-Player        Python   
439    499    ProPanek/PlanningPoker    JavaScript   

                                       readme_contents  \
0    Deep mind pokerbot for pokerstars, partypoker ...   
1    # Pokémon Red and Blue [![Build Status][ci-bad...   
2    # PyPokerEngine\n\n[![Build Status](https://tr...   
3    PokerStove\n==========\n\n[![OpenHUB Metrics](...   
4    Poker framework for Python\n==================...   
..                                                 ...   
435  ### 满意请给个star\n### [1.0原版js](https://github.co...   
436  This [Actionscript Flash poker (5-card & Texas...   
437  # pokerserver\n[![Codacy Badge](https://api.co...   
438  # AI Poker Player\n===========================...   
439  # Planning Poker\n[![Codacy Badge](https://api...   

                                               content  \
0    Deep mind pokerbot for pokerstars, partypoker ...   
1    # Pokémon Red and Blue [![Build Status][ci-bad...   
2    # PyPokerEngine\n\n[![Build Status](https://tr...   
3    PokerStove\n==========\n\n[![OpenHUB Metrics](...   
4    Poker framework for Python\n==================...   
..                                                 ...   
435  ### 满意请给个star\n### [1.0原版js](https://github.co...   
436  This [Actionscript Flash poker (5-card & Texas...   
437  # pokerserver\n[![Codacy Badge](https://api.co...   
438  # AI Poker Player\n===========================...   
439  # Planning Poker\n[![Codacy Badge](https://api...   

                                       stemmed_content  \
0    deep mind pokerbot pokerstar partypok ggpoker ...   
1    pokemon red blue build statuscibadgeci disasse...   
2    pypokerengin build statushttpstravisciorgishik...   
3    pokerstov openhub metricshttpswwwopenhubnetppo...   
4    poker framework python travi coveral python re...   
..                                                 ...   
435  star 10jshttpsgithubcomimgamedesignerpokergame...   
436  actionscript flash poker 5card texa holdemhttp...   
437  pokerserv codaci badgehttpsapicodacycomproject...   
438  ai poker player initi commit ai poker player s...   
439  plan poker codaci badgehttpsapicodacycomprojec...   

                                        lemmed_content  
0    deep mind pokerbot pokerstars partypoker ggpok...  
1    pokemon red blue build statuscibadgeci disasse...  
2    pypokerengine build statushttpstravisciorgishi...  
3    pokerstove openhub metricshttpswwwopenhubnetpp...  
4    poker framework python travis coverall python ...  
..                                                 ...  
435  star 10jshttpsgithubcomimgamedesignerpokergame...  
436  actionscript flash poker 5card texas holdemhtt...  
437  pokerserver codacy badgehttpsapicodacycomproje...  
438  ai poker player initial commit ai poker player...  
439  planning poker codacy badgehttpsapicodacycompr...  

[440 rows x 7 columns]

In [11]:
labeled_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 440 entries, 0 to 439
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   index            440 non-null    int64 
 1   repo             440 non-null    object
 2   language         440 non-null    object
 3   readme_contents  440 non-null    object
 4   content          440 non-null    object
 5   stemmed_content  440 non-null    object
 6   lemmed_content   440 non-null    object
dtypes: int64(1), object(6)
memory usage: 27.5+ KB


In [12]:
# labeled_df is the final DF, and I dropped 60 columns due to nulls and duplicate repo issues. This final DF will be used for the explore section
# lets take a look at overall composition by programming language:

def show_counts_and_ratios(df, column):
    """
    Takes in a dataframe and a string of a single column
    Returns a dataframe with absolute value counts and percentage value counts
    """
    labels = pd.concat([df[column].value_counts(),
                    df[column].value_counts(normalize=True)], axis=1)
    labels.columns = ['n', 'percent']
    labels
    return labels

show_counts_and_ratios(labeled_df, "language")


,n,percent
JavaScript,98,0.222727
Python,81,0.184091
Java,53,0.120455
C++,28,0.063636
TypeScript,24,0.054545
...,...,...
Julia,1,0.002273
Elm,1,0.002273
Apex,1,0.002273
Pascal,1,0.002273


In [13]:
ADDITIONAL_STOPWORDS = []

def clean(text):
        'A simple function to cleanup text data'
        wnl = nltk.stem.WordNetLemmatizer()
        stopwords = nltk.corpus.stopwords.words('english') + ADDITIONAL_STOPWORDS
        text = (unicodedata.normalize('NFKD', text)
                .encode('ascii', 'ignore')
                .decode('utf-8', 'ignore')
                .lower())
        words = re.sub(r'[^\w\s]', '', text).split()
        return [wnl.lemmatize(word) for word in words if word not in stopwords]


In [14]:
languages = labeled_df.language.unique()
language_df = {}

for lang in languages:
    language_df[lang] = labeled_df[labeled_df.language == lang].lemmed_content
    language_df[lang] = pd.Series(" ".join(language_df[lang].values).split(' '), name=lang).value_counts()

language_df["Python"]









game            267
poker           232
player          231
hand            200
card            169
               ... 
rival             1
pipinstalled      1
boolean           1
switching         1
midsund           1
Name: Python, Length: 5161, dtype: int64

In [15]:
python

NameError: name 'python' is not defined

In [ ]:
labeled_df.isna().sum()

In [ ]:
labeled_df.head(1)

In [ ]:
# def remove_stopwords(string, extra_words = [], exclude_words = []):
#     additional_stopwords = ['github', 'http', 'code']
#     nltk.download('wordnet')
#     nltk.download('stopwords')
#     stopword_list = stopwords.words('english') + additional_stopwords
#     stopword_list = set(stopword_list) - set(exclude_words)
#     stopword_list = stopword_list.union(set(extra_words))
#     words = string.split()
#     filtered_words = [word for word in words if word not in stopword_list]
#     string_without_stopwords = ' '.join(filtered_words)
#     return string_without_stopwords